In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!jupyter serverextension enable --sys-prefix voila

In [3]:
human_types = ['man','woman','boy','girl']

In [4]:
path = Path()

In [5]:
learn = load_learner(path/'human_classifier.pkl')

In [6]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl.clear_output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [7]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]: .04f}'

In [8]:
btn_run.on_click(on_click_classify)

In [9]:
VBox([widgets.Label('Select your human'),btn_upload, btn_run, out_pl, lbl_pred])

D:\MiscPrograms\Miniconda\envs\fastai_v2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
